## Pull Packages

In [0]:
import polars as pl
import polars.selectors as cs

from wadoh_raccoon.utils import helpers
from wadoh_subtyping import transform as tf, qa

from pydantic import BaseModel

In [0]:
# create the types
class PHLTables(BaseModel):
    phl_df: pl.DataFrame 
    received_submissions_df: pl.DataFrame
    base_cols: list

    model_config = {
        "arbitrary_types_allowed": True
    }

# create the types
class WDRSTables(BaseModel):
    respnet_investigation: pl.DataFrame 
    respnet_wizard: pl.DataFrame

    model_config = {
        "arbitrary_types_allowed": True
    }


In [0]:
def phl_tables() -> PHLTables:
    """ Read PHL

    Description
    -----------
    To be run inside databricks

    """
    spark_df = spark.sql(
        """
        SELECT * 
        FROM `01_bronze_prod`.comp_data.vz_epi_micro_virology_influenza
        WHERE SpecimenDateCollected > '2024-01-01'
        """
    )

    phl = pl.from_pandas(spark_df.toPandas())

    base_cols = ["submission_number", "internal_create_date"] + phl.columns 

    # make received submissions df
    # useful for collecting all submissions in the same format
    received_submissions_df = (
        helpers.save_raw_values(df_inp=phl,primary_key_col="PHLAccessionNumber")
    )

    # unnest it to get the submission_number
    # subset out records from df_to_process_inp with submission number in list_to_fuzzy, assign to submissions_to_fuzzy_df
    phl_df = (
        received_submissions_df
        .unnest(pl.col('raw_inbound_submission'))
        .select(sorted(base_cols))
    )

    pt = PHLTables(
        phl_df=phl_df,
        received_submissions_df=received_submissions_df,
        base_cols=base_cols
    )

    return pt

In [0]:
pt = phl_tables()

In [0]:
pt.phl_df.head()

In [0]:
def wdrs_tables() -> WDRSTables:
    """ Pull WDRS Tables

    Usage
    -----
    Run this inside databricks
    """

    spark_resp_inves = spark.sql(
        """
        SELECT *
        FROM fc_bronze_wdrs_prod.dbo.dd_gcd_respnet_investigation
        WHERE CODE = 'FLUHP';
        """
    )

    spark_resp_wiz = spark.sql(
        """
        SELECT *
        FROM fc_bronze_wdrs_prod.dbo.dd_gcd_respnet_wizard
        WHERE CODE = 'FLUHP';
        """
    )

    respnet_investigation = pl.from_pandas(spark_resp_inves.toPandas())

    respnet_wizard = pl.from_pandas(spark_resp_wiz.toPandas())

    wt = WDRSTables(
        respnet_investigation=respnet_investigation,
        respnet_wizard=respnet_wizard
    )

    return wt

In [0]:
wt = wdrs_tables()

In [0]:
respnet = (
    wt.respnet_wizard
    .join(wt.respnet_investigation,how="left",on='CASE_ID')
    .rename({'SPECIMEN__COLLECTION__DTTM': 'SPECIMEN_COLLECTION_DTTM'})
)